## HW 5: Choose your own adventure.
The goal of this last homework is to capitalize on the skills you have developed to perform a more open ended analysis of a dataset. To complete this HW you only need to CHOOSE ONE of the three prompts below and make an attempt.  Do your best, justify you answer to the question posed in each case using what we have learned during the quarter. Some problems are more open ended then others as noted in the prompts.  Choose your problem according to your skills and motivations.  30 pts total.

### Option 1: predicting phenotype with random forest regression.
Paper: Zeqian Li, Ahmed Selim, Seppe Kuehn. PLoS Comp Biol. 2023. https://doi.org/10.1371/journal.pcbi.1011705

**I expect this to be straightforward. The main challenge will be getting familiar with setting up and running random forest regression. The data is already in a form that should be easy to work with.**

In this paper we took ~100 different bacterial isolates and for each one we measured whether or not it could grow using 10 different carbon sources.  The experiment was to put them in media conditions with each carbon source in isolation, wait a couple of days, and measure if any growth had occurred.  Then we wanted to see if we could predict this binary (growth/no-growth) phenotype from the genes (presence/absence) that each strain possessed. We ended up doing this regression using random forests. I want you to try and recapitulate a random forest regression using these data.

The growth measurements are in the zeqian_growth_data_final.csv (call this matrix P)
* rows are strains
* columns are carbon sources

The gene presence/absence matrix is in zeqian_ko_data_final.csv  (call this matrix G)
* rows are strains
* columns are genes designated by KEGG Orthology groups (https://www.genome.jp/kegg/)

**First** What I want you to do is pick a column of P (any one) and regress it on ALL columns of G using a random forest regression.  So use the presence and absence of all the genes when trying to predict P. Note, use the built in Python regression package sklearn.ensemble.RandomForestRegressor.

**Second** I want you to assess the out of sample predictive power of the model using cross validation.  Does it work?

**Third** Now go look carefully at the paper.  When you chose your test set for CV how did you select the strains (rows) to hold out?  Why do you think the regression is working?


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn

matrix_p = pd.read_csv('zeqian_growth_data_final.csv')
matrix_g = pd.read_csv('zeqian_ko_data_final.csv')

matrix_p
#matrix_g

#we are going to check mannose

,strain,Arabinose,Butyrate,Deoxyribose,Glucuronic acid,Glycerol,Mannitol,Mannose,Melibiose,Propionate,Raffinose
0,HMWF001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HMWF003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HMWF005,1.0,NaN,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
3,HMWF006,1.0,NaN,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
4,HMWF008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
91,sif2233,1.0,NaN,0.0,NaN,1.0,1.0,1.0,0.0,NaN,0.0
92,sif2332,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
93,sif2416,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
94,sif2431,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [120]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.model_selection import train_test_split


In [121]:
mat_p_proc = matrix_p.iloc[:,7]
mat_g_proc = matrix_g.iloc[:,1:]

nan_rows = mat_p_proc[mat_p_proc.isnull()]
print(nan_rows)

mat_p_proc = mat_p_proc.drop([12,15,34,49,54,63])
mat_g_proc = mat_g_proc.drop([12,15,34,49,54,63])
print(mat_g_proc)


rf = RandomForestClassifier()
#print(matrix_g.iloc[:,1:])
rf.fit(mat_g_proc, mat_p_proc)

12   NaN
15   NaN
34   NaN
49   NaN
54   NaN
63   NaN
Name: Mannose, dtype: float64
    K00001  K00003  K00004  K00005  K00007  K00008  K00009  K00010  K00012  \
0        0       0       0       0       0       0       0       0       1   
1        0       0       0       0       0       0       0       0       1   
2        0       1       0       0       0       0       0       0       1   
3        0       1       0       0       0       0       0       0       1   
4        0       1       0       0       0       0       0       1       1   
..     ...     ...     ...     ...     ...     ...     ...     ...     ...   
91       0       1       0       0       1       0       0       0       0   
92       0       1       0       0       0       0       0       0       0   
93       1       0       0       0       0       0       1       0       1   
94       0       1       0       1       0       0       0       0       1   
95       1       0       0       0       0       0       1

RandomForestClassifier()

In [122]:
from sklearn.model_selection import cross_val_score
#interesting function, not entirely sure how it works though
print(np.mean(cross_val_score(rf, mat_g_proc, mat_p_proc, cv=8)))

print('manual cv')
#more manual cross validation
for x in range(10):
    X_train, X_test, y_train, y_test = train_test_split(
        mat_g_proc, mat_p_proc, test_size=0.4, random_state=0)

    rf2 = RandomForestClassifier()
    rf2.fit(X_train, y_train)

    print(rf2.score(X_test, y_test))

0.8674242424242424
manual cv
0.8888888888888888
0.8333333333333334
0.8611111111111112
0.8333333333333334
0.8888888888888888
0.8611111111111112
0.8333333333333334
0.8333333333333334
0.8055555555555556
0.8611111111111112


ANSWERS:The out of fold score is between 0.8 and 0.9, which is comfortably close to a perfect score of 1, indicating that it works. When using a 8-fold cross validation, the score can get as high as 0.93. 

In terms of preprocessing, I chose to omit rows that had a missing value for y, or ability to utilize mannose, since it wouldn't be helpful to train on a datapoint that is missing a value for the output datapoint. 
For the cross validation, the partition of points that were taken out to serve as the train and test were randomly chosen, with the amount partitioned to train and test being chosen by me. The regression is working when these points are randomly chosen because the random forest model is actually making a correlation between the phylogeny(determined by the gene similarity) and the trait(ability to utilize mannose). This is corroborated by the paper, which finds that when the paritions are chosen out of clade, the models predictive power fails, indicating when the model is only trained on certain phylogenetic groups of strains its ability to make predicitions for other unrelated phylogenetic groups is poor.

### Option 2: See if you can improve on a linear regression using a neural network.

**This will be more challenging since we did not actively work with neural networks. I cannot independently confirm whether nor not an NN will outperform our linear regression or not. However, the data wrangling aspects of this option were already done in project 1.  So that part is easy.**

For the regressions we performed in project 1 using gene presence/absence to predict phenotypes using linear regression.  Can you improve the **out of sample** predictions using a neural network?

What I want to you to do is take the regression for $r_A$ based on gene presence/absence ($x_{i,j}$) and see if you can improve.  Recall that last time we fit a model of the form:

$$ r_A^i = \beta_0 + \sum_j \beta_j x_{i,j}$$

Instead, I want you to use a neural network to predict $$r_A^i = f(x_{i,j})$$ where the function $f$ is a neural network.  I would recommend using a multilayer perceptron feedforward neural network.  You will need to use regularization to avoid overfitting either via what is called 'dropout', early stopping during training, or (easiest) just regularization on the weights in the network. This works in the same way that the L1 penalty works for LASSO, but is applied to network weights. The one other key tweak is to make the final node of the network have a linear activation function rather than RelU. Note the number of input nodes should be the number of independent variables.  To do this I recommend using TensorFlow and Keras which are the standard machine learning NN platforms for python.  Keras is a high level package that sits on top of TensorFlow and uses it to setup NN models.


In [3]:
# Load the Excel file into a pandas DataFrame
import numpy as np
import pandas as pd
file_path = 'DataMatrix_Project1.xlsx'
dataframe = pd.read_excel(file_path)


# Filter the rows where the second column ('phenotype') includes "NAR"
nar_rows = dataframe[dataframe['phenotype'].str.contains("NAR")]

# Correctly select the columns 2 to 19 (which are indices 1 through 18 in zero-based Python indexing)
XA = nar_rows.iloc[:, 2:19].to_numpy()
YRA = nar_rows['YRA'].to_numpy()
YGA = nar_rows['YGA'].to_numpy()

nir_rows = dataframe[dataframe['phenotype'].str.contains("NIR")]
XI = nir_rows.iloc[:,2:19].to_numpy()
YRI = nir_rows['YRI'].to_numpy()
YGI = nir_rows['YGI'].to_numpy()

## Option 3: Dimension reduction on the global ocean microbiome.

**This option is technically easier than option 2, but harder than option 1. In particular, it requires some relatively messy data wrangling followed by some relatively simple data analysis.**

The question is based on this 2015 paper on the global ocean microbiome.https://www.science.org/doi/epdf/10.1126/science.1261359

The dataset constructed here are measurements of taxonomic diversity for >100 samples from the ocean microbiome at various depths. For each sample they perform sequencing measurements and they also perform environmental measurements (temperature, nutrient levels, etc).  The goal here is to perform PCA on the taxonomic data and asnwer the following questions:

1. Is there any low dimensional structure in a matrix that has samples as rows and abundances of taxa as columns?  That is, how much variance in the data is explained by the first couple PCs?
2. What environmental variable, if any, is most strongly correlated with the varaition in abundances you characterized in question 1?  That is, what environmental parameter correlates most strongly with sample projections on the first PC?

In [ ]:
# Code to load the taxonomic data! 
D = pd.read_csv('miTAG.taxonomic.profiles.release.tsv', sep='\t')
D.columns


The columns of D are samples, the rows are taxa.  Note that some columns contain the string '0.22_1.6' -- and others contain '0.22_3' these refer to size fractions.  For the former, only particles in the size range of 0.22um to 1.6um were sequenced, in the latter, 0.22um to 3um.  **Restrict your analysis to the larger size fraction -- 0.22um to 3um**.

In [ ]:
# Here is code to load the environmental data and associated sample IDs
EnvData = pd.read_excel('OM.CompanionTables.xlsx',sheet_name = 'Table W8')
SampleIDs = pd.read_excel('OM.CompanionTables.xlsx',sheet_name = 'Table W1')

Here is the data wrangling problem. 

EnvData has the samples as rows and the environmental parameters as columsn. The first column of EnvData contains a "Pangaea Sample ID" which is NOT the same as the sample names (columns) in the dataframe (D) loaded above that contains the taxonomic information. The second dataframe loaded above (SampleIDs) contains a column called "Sample label" that DOES match the column names in D above. It also contains a column labeled "PANGAEA sample identifier".  Therefore, using the SampleIDs dataframe you can match a Sample ID (column names of D) to a Pangaea label. From this matching you can then associate a row of EnvData with each samples taxonomic data.  Do this carefully and make sure you manually check a few to make sure.  You will need to do this to answer question 2!